# Exploratory Analysis of data

This notebook examins the time patterns, seasonal effects, correlations among pollutants, and relationships between meterological and chemical variables in the Air Quality dataset from UCI.



## Setup

In [ ]:
import pandas as pd

In [20]:
path_to_data = "../../dataset/raw/AirQualityUCI.csv"

df = pd.read_csv(
    path_to_data,
    sep=';',       
    decimal=','     
)

df.head()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN
3,10/03/2004,21.00.00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,NaN,NaN
4,10/03/2004,22.00.00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,NaN,NaN


# Data quality analysis

## Missing values
Firstly, observe that the dataset contains sentinel values `-200` as a placeholder for missing values (See row 9 above).

This will need to be treated as missing data appropriately using `NaN` during data preprocessing, otherwise statistical analysis (e.g finding mean) will be skewed and machine learning models would be factoring in incorrect data during their training process.



In [21]:
sentinel_mask = (df == -200.0)
sentinel_counts = sentinel_mask.sum()

print("Number of Sentinel values per variable")
print(sentinel_counts[sentinel_counts > 0].sort_values(ascending=False))

Number of Sentinel values per variable
NMHC(GT)         8443
CO(GT)           1683
NO2(GT)          1642
NOx(GT)          1639
PT08.S1(CO)       366
C6H6(GT)          366
PT08.S2(NMHC)     366
PT08.S3(NOx)      366
PT08.S4(NO2)      366
PT08.S5(O3)       366
T                 366
RH                366
AH                366
dtype: int64


* We can see that `NMHC(GT)` is missing 8443 values (~90% of total data), thus it is unusable as a target variable for our models and will need to be removed during processing.
  
*  Relative pollutants such as `CO(GT)`, `NO2(GT)` `NOx(GT)` are missing a number of values between [1600, 1700] (~17% of total data). 
  
*  Finally, sensors and meterology `PT08.S` series, `T`, `RH`, `AH` show a gap of exactly 366 sentinels (~4% of total data).
  
The fact that 10 columns share the exact same count of sentinel values (366) strongly suggests that these gaps are not random, rather it may be caused by synchronized system downtime or data logging failure. This is confirmed by observing entries in the dataset where many variables are missing values, shown below.

In [29]:
df.iloc[6700:6710]

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
6700,14/12/2004,22.00.00,2.5,-200.0,-200.0,-200.0,-200.0,341.0,-200.0,112.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6701,14/12/2004,23.00.00,2.1,-200.0,-200.0,-200.0,-200.0,365.0,-200.0,108.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6702,15/12/2004,00.00.00,2.9,-200.0,-200.0,-200.0,-200.0,510.0,-200.0,115.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6703,15/12/2004,01.00.00,2.3,-200.0,-200.0,-200.0,-200.0,352.0,-200.0,96.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6704,15/12/2004,02.00.00,1.7,-200.0,-200.0,-200.0,-200.0,299.0,-200.0,87.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6705,15/12/2004,03.00.00,0.9,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6706,15/12/2004,04.00.00,0.7,-200.0,-200.0,-200.0,-200.0,103.0,-200.0,61.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6707,15/12/2004,05.00.00,0.8,-200.0,-200.0,-200.0,-200.0,133.0,-200.0,61.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6708,15/12/2004,06.00.00,1.0,-200.0,-200.0,-200.0,-200.0,244.0,-200.0,82.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN
6709,15/12/2004,07.00.00,1.2,-200.0,-200.0,-200.0,-200.0,222.0,-200.0,83.0,-200.0,-200.0,-200.0,-200.0,-200.0,NaN,NaN


## Trailing rows and columns

The dataset contains trailing, empty rows and columns that need to be stripped out in pre-processing. 
* Empty rows begin at `9538` and end at `9470`, meaning we need to strip the last 112 rows from the dataset. 
* Columns `15` and `16` are also empty and need to be stripped.

In [28]:
df.tail()

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
9466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Comma Decimal separator
The raw data uses commas to mark decimal places instead of `.`. If not addressed, this makes the columns non-numeric, breaking math operations.

# Analysis of Relationships and Correlations

## Time patterns